In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')


!nvidia-smi

Mounted at /content/drive
Sat Apr 27 21:35:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
!pip install datasets==2.11.0 --quiet
!pip install accelerate  --quiet
!pip install evaluate   --quiet
!pip install rouge_score --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn
from tqdm import tqdm
from tqdm.auto import trange
import torch
from datasets import Dataset

In [ ]:
def load(repo_id, load_in_8bit=False):
    return (
        AutoModelForCausalLM.from_pretrained(
            repo_id, device_map='auto', load_in_8bit=load_in_8bit
        ),
        AutoTokenizer.from_pretrained(repo_id)
    )

teacher, tokenizer = load("EleutherAI/pythia-1b", False)
tokenizer.pad_token = tokenizer.eos_token
teacher = teacher.to(torch.bfloat16).cuda()

student = load("EleutherAI/pythia-70m")[0]

student = student.cuda().to(torch.bfloat16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
csv_path = "/content/drive/MyDrive/project/Jfleg4-2-4.csv"
df = pd.read_csv(csv_path)
dataset = Dataset.from_pandas(df)
split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()
test_df = test_df.head(100)

In [ ]:
lr = 1e-4
train_max_length = 1024
batch_size = 4
grad_acc_steps = 128
total_steps = 256
distil_temp = 1.3
alpha = 0.5

optim = torch.optim.AdamW(student.parameters(), lr)
scheduler = torch.optim.lr_scheduler.LinearLR(optim, start_factor=1, end_factor=0.1, total_iters=total_steps)
logit_loss_fn = nn.KLDivLoss(reduction="batchmean")
logit_accumulator = 0
loss_accumulator = 0

def get_next():
    global next_index
    input_text = train_df['input'].iloc[next_index]
    target_text = train_df['target'].iloc[next_index]
    next_index = (next_index + 1) % len(train_df)
    return input_text, target_text

next_index = 0

for i in trange(total_steps*grad_acc_steps):
    text = [get_next() for i in range(batch_size)]
    losses = {}
    tokenized = {
        k:v.cuda() for k,v in tokenizer(
            [t[0] for t in text], return_tensors="pt", truncation=True, padding=True
        ).items()
    }
    input_ids = tokenized['input_ids'][...,:train_max_length]
    attention_mask = tokenized['attention_mask'][...,:train_max_length]
    targ_logits = teacher(input_ids).logits.clone().detach()
    student_out = student(input_ids, labels=input_ids)
    logits = student_out.logits
    loss = student_out.loss
    s_logits = logits
    t_logits = targ_logits
    mask = attention_mask.unsqueeze(-1).expand_as(s_logits)
    s_logits_slct = s_logits.view(-1, s_logits.size(-1))
    t_logits_slct = t_logits.view(-1, s_logits.size(-1))
    loss_ce = (
        logit_loss_fn(
            nn.functional.log_softmax(s_logits_slct / distil_temp, dim=-1),
            nn.functional.softmax(t_logits_slct / distil_temp, dim=-1),
        )
        * (distil_temp) ** 2
    )
    logit_loss = loss_ce
    ((loss*(1-alpha) + logit_loss*alpha) / grad_acc_steps).backward()
    loss_accumulator+=loss.item()/grad_acc_steps
    logit_accumulator+=logit_loss.item()/grad_acc_steps
    if (i+1)%grad_acc_steps==0:
        optim.step()
        optim.zero_grad()
        scheduler.step()
        losses['Student Loss'] = loss_accumulator
        losses['Student Perplexity'] = torch.exp(
            torch.tensor(loss_accumulator)
        ).item()
        losses['Distillation Loss'] = logit_accumulator
        losses['Learning Rate'] = scheduler.get_last_lr()[0]
        print(losses)
        loss_accumulator = 0
        logit_accumulator = 0
# Save the model
model_save_path = '/content/drive/MyDrive/project/model/distillpythia'
tokenizer_save_path = '/content/drive/MyDrive/project/model/tokenizer/distillpythia'
student.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

  0%|          | 0/32768 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'Student Loss': 7.656494140625, 'Student Perplexity': 2114.331787109375, 'Distillation Loss': 3.48095703125, 'Learning Rate': 9.96484375e-05}
{'Student Loss': 5.3211669921875, 'Student Perplexity': 204.62254333496094, 'Distillation Loss': 4.052490234375, 'Learning Rate': 9.9296875e-05}
{'Student Loss': 5.15234375, 'Student Perplexity': 172.8361053466797, 'Distillation Loss': 4.275146484375, 'Learning Rate': 9.894531250000001e-05}
{'Student Loss': 4.86376953125, 'Student Perplexity': 129.511474609375, 'Distillation Loss': 4.2073974609375, 'Learning Rate': 9.859375000000001e-05}
{'Student Loss': 4.85546875, 'Student Perplexity': 128.44088745117188, 'Distillation Loss': 4.1087646484375, 'Learning Rate': 9.824218750000001e-05}
{'Student Loss': 4.9951171875, 'Student Perplexity': 147.69024658203125, 'Distillation Loss': 3.9439697265625, 'Learning Rate': 9.789062500000001e-05}
{'Student Loss': 5.0013427734375, 'Student Perplexity': 148.61257934570312, 'Distillation Loss': 3.865234375, 'Lear

('/content/drive/MyDrive/project/model/tokenizer/distillpythia/tokenizer_config.json',
 '/content/drive/MyDrive/project/model/tokenizer/distillpythia/special_tokens_map.json',
 '/content/drive/MyDrive/project/model/tokenizer/distillpythia/tokenizer.json')

In [ ]:
from transformers import pipeline
model_save_path = '/content/drive/MyDrive/project/model/distillpythia'
tokenizer_save_path = '/content/drive/MyDrive/project/model/tokenizer/distillpythia'
model = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)
predictor = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

batch_size = 16

results = {'Input Text': [], 'Predicted Text': [], 'Correct Text': []}

for i in tqdm(range(0, len(test_df), batch_size)):
    batch = test_dataset[i:i + batch_size]
    input_texts = batch['input']
    correct_texts = batch['target']

    generated_outputs = predictor(input_texts, max_length=1024, truncation=True)
    # print(generated_outputs)
    predicted_texts = [output[0]['generated_text'] for output in generated_outputs]

    results['Input Text'].extend(input_texts)
    results['Predicted Text'].extend(predicted_texts)
    results['Correct Text'].extend(correct_texts)

df_results = pd.DataFrame(results)
df_results.to_csv('/content/drive/MyDrive/project/dataset_dp.csv', index=False)
print("Predictions have been saved to CSV.")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 7/7 [00:41<00:00,  5.89s/it]


Predictions have been saved to CSV.


In [ ]:
import evaluate

df = pd.read_csv('/content/drive/MyDrive/project/dataset_dp.csv')


dp_predicted_text_list = df['Predicted Text'].tolist()
actual_test_list = df['Correct Text'].tolist()

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

print("Knowledge Distillation Predictions BLEU Results:")
base_bleu_results = bleu.compute(predictions=dp_predicted_text_list, references=[[ref] for ref in actual_test_list])
print(base_bleu_results)

print("Knowledge Distillation Predictions ROUGE Results:")
base_rouge_results = rouge.compute(predictions=dp_predicted_text_list, references=actual_test_list)
print(base_rouge_results)

Knowledge Distillation Predictions BLEU Results:
{'bleu': 0.3068644189238982, 'precisions': [0.38425171177045975, 0.32813018930587845, 0.28375634517766496, 0.2478440841669541], 'brevity_penalty': 1.0, 'length_ratio': 2.283693224125093, 'translation_length': 6134, 'reference_length': 2686}
Knowledge Distillation Predictions ROUGE Results:
{'rouge1': 0.8349688154214581, 'rouge2': 0.7293935945651266, 'rougeL': 0.8287641075828661, 'rougeLsum': 0.8277915986128987}
